In [4]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit

import numpy as np 
print(rdkit.__version__)

2022.03.5


In [5]:
import os 

ms = []
names = []

for dir in os.listdir('./data'):
    if os.path.isdir('./data/'+dir):
        for file in os.listdir('./data/'+dir):
            if file.endswith('.pdb'):
                names.append(dir)
                m = rdkit.Chem.rdmolfiles.MolFromPDBFile('./data/'+dir+"/"+file)
                if m is None:
                    print("Error reading file:", file)
                else:
                    ms.append(m)
                    #print(type(m))


Error reading file: struc.pdb


[11:24:55] WARNING: not removing hydrogen atom without neighbors
[11:24:55] Explicit valence for atom # 0 Cl, 2, is greater than permitted


In [24]:

from rdkit import DataStructs

print("We have " , len(ms), " molecules ")

for fingersize in [8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]:

    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=fingersize)
    rdgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=fingersize)
    apgen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize=fingersize)
    ttgen = rdFingerprintGenerator.GetTopologicalTorsionGenerator(fpSize=fingersize)

    mf_maxnonzero = 0
    rd_maxnonzero = 0
    ap_maxnonzero = 0
    tt_maxnonzero = 0
    for m in ms:
        mf_np_bits = mfgen.GetFingerprintAsNumPy(m)
        mf_np_counts = mfgen.GetCountFingerprintAsNumPy(m)

        rd_np_bits = rdgen.GetFingerprintAsNumPy(m)
        rd_np_counts = rdgen.GetCountFingerprintAsNumPy(m)

        ap_np_bits = apgen.GetFingerprintAsNumPy(m)
        ap_np_counts = apgen.GetCountFingerprintAsNumPy(m)

        tt_np_bits = ttgen.GetFingerprintAsNumPy(m)
        tt_np_counts = ttgen.GetCountFingerprintAsNumPy(m)

        # print non zero value and posizion in the array
        #print(np.nonzero(mf_np_bits), mf_np_bits[np.nonzero(np_bits)])
        #print(np.nonzero(mf_np_counts), mf_np_counts[np.nonzero(np_counts)])

        if np.count_nonzero(mf_np_bits) > mf_maxnonzero:
            mf_maxnonzero = np.count_nonzero(mf_np_bits)
        if np.count_nonzero(mf_np_counts) > mf_maxnonzero:
            mf_maxnonzero = np.count_nonzero(mf_np_counts)

        if np.count_nonzero(rd_np_bits) > rd_maxnonzero:
            rd_maxnonzero = np.count_nonzero(rd_np_bits)
        if np.count_nonzero(rd_np_counts) > rd_maxnonzero:
            rd_maxnonzero = np.count_nonzero(rd_np_counts)

        if np.count_nonzero(ap_np_bits) > ap_maxnonzero:
            ap_maxnonzero = np.count_nonzero(ap_np_bits)
        if np.count_nonzero(ap_np_counts) > ap_maxnonzero:
            ap_maxnonzero = np.count_nonzero(ap_np_counts)

        if np.count_nonzero(tt_np_bits) > tt_maxnonzero:
            tt_maxnonzero = np.count_nonzero(tt_np_bits)
        if np.count_nonzero(tt_np_counts) > tt_maxnonzero:
            tt_maxnonzero = np.count_nonzero(tt_np_counts)                

    print(fingersize, "Max non zero value in FPs is: ", mf_maxnonzero, 
          rd_maxnonzero, ap_maxnonzero, tt_maxnonzero)


We have  151  molecules 
8 Max non zero value in FPs is:  6 7 5 4
16 Max non zero value in FPs is:  7 10 6 4
32 Max non zero value in FPs is:  7 13 6 4
64 Max non zero value in FPs is:  7 16 6 4
128 Max non zero value in FPs is:  7 17 6 4
256 Max non zero value in FPs is:  7 17 6 4
512 Max non zero value in FPs is:  7 18 6 4
1024 Max non zero value in FPs is:  7 18 6 4
2048 Max non zero value in FPs is:  7 18 6 4
4096 Max non zero value in FPs is:  8 18 6 4
